In [1]:
from pyCHNOSZ import subcrt

In [8]:
output = subcrt(["H2", "O2", "H2O"], [-1.0, -0.5, 1.0], ["aq", "gas", "liq"],
                T=[30, 50], P=100, IS=0.1)

subcrt: 3 species at 2 values of T (ºC) and P (bar) (wet) [energy units: cal]

nonideal: calculations for H2 (Setchenow equation)



In [9]:
output["reaction"]

,coeff,name,formula,state,ispecies
62,-1.0,H2,H2,aq,62.0
2612,-0.5,oxygen,O2,gas,2612.0
1,1.0,water,H2O,liq,1.0


In [10]:
output["out"]

,T,P,rho,logK,G,H,S,V,Cp,loggam,IS
1,30.0,100,1.000017,43.854305,-60831.295981,NaN,NaN,-7.494052,NaN,0.000782,0.1
2,50.0,100,0.992305,40.833637,-60378.106821,NaN,NaN,-8.259704,NaN,0.000782,0.1


In [ ]:
import os
os.environ["R_HOME"] = "/opt/conda/envs/python3andR/lib/R/"
import rpy2.robjects as ro

In [ ]:
r = ro.r
r.source('test.r')

In [ ]:
from rpy2.robjects.packages import importr
CHNOSZ = importr("CHNOSZ")

In [ ]:
CHNOSZ.info("pyrite")

In [ ]:
ro.Vector(["H2", "O2", "H2O"])

In [ ]:
ro.Vector([-1.0, -0.5, 1.0])

In [ ]:
CHNOSZ.subcrt(species=ro.Vector(["H2", "O2", "H2O"]),
              coeff=ro.FloatVector([-1.0, -0.5, 1.0]))

In [ ]:
a = CHNOSZ.subcrt(species=ro.Vector(["H2", "O2", "H2O"]),
                       coeff=ro.FloatVector([-1.0, -0.5, 1.0]))
a

In [ ]:
a[0]

In [ ]:
import os
os.environ["R_HOME"] = "/opt/conda/envs/python3andR/lib/R/"
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

In [ ]:
CHNOSZ = importr("CHNOSZ")

In [ ]:
pd_df = pd.DataFrame({'int_values': [1,2,3],
                      'str_values': ['abc', 'def', 'ghi']})

pd_df

In [ ]:
# convert pandas to R dataframe

import numpy as np
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# Create pandas df
df = pd.DataFrame( np.random.randn(5,2), # 5 rows, 2 columns
               columns = ["A","B"], # name of columns
               index = ["Max", "Nathy", "Tom", "Joe", "Kathy"] ) # does this index work?

# Convert pandas to r
r_df = pandas2ri.py2ri(df)
type(r_df)

In [ ]:
from rpy2.robjects import pandas2ri

pd_df = pandas2ri.ri2py_dataframe(a[0])
pd_df

In [ ]:
base = importr('base')

In [ ]:
for warning in base.warnings():
    print(warning)

In [ ]:
base.warnings()[0]

In [ ]:
args = {'species':"CH4", 'T':ro.FloatVector([20, 30])}
CHNOSZ.subcrt(**args)

In [ ]:
### Filter rpy2 runtime warnings
# import warnings
# from rpy2.rinterface import RRuntimeWarning
# warnings.filterwarnings("ignore", category=RRuntimeWarning)

In [ ]:
import warnings
from rpy2.rinterface import RRuntimeWarning

with warnings.catch_warnings():
    warnings.filterwarnings(
        action='ignore',
        category=RRuntimeWarning,
        message='test warning message')
    
    args = {'species':"CH4", 'T':ro.FloatVector([20, 30])}
    CHNOSZ.subcrt(**args)

In [ ]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings(record=True):
    warnings.simplefilter("ignore")
    fxn()
warnings.showwarning()

In [ ]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings(record=True) as w:
    # Cause all warnings to always be triggered.
    warnings.simplefilter("always")
    # Trigger a warning.
    fxn()
    # Verify some things
#     assert len(w) == 1
#     assert issubclass(w[-1].category, DeprecationWarning)
#     assert "deprecated" in str(w[-1].message)
print(w[0].message)

In [ ]:
import os
os.environ["R_HOME"] = "/opt/conda/envs/python3andR/lib/R/"
from rpy2.rinterface import RRuntimeWarning

In [ ]:
import os
os.environ["R_HOME"] = "/opt/conda/envs/python3andR/lib/R/"
import warnings
from rpy2.rinterface import RRuntimeWarning

with warnings.catch_warnings():
    warnings.filterwarnings(
        action='ignore',
        category=RRuntimeWarning,
        message='Loading rpy2 R environment and CHNOSZ')
    
    import rpy2.robjects as ro
    from rpy2.robjects.packages import importr
    
    CHNOSZ = importr("CHNOSZ")

 ```
  subcrt(species, coeff = 1, state = NULL,
    property = c("logK","G","H","S","V","Cp"),
    T = seq(273.15,623.15,25), P = "Psat", grid = NULL, 
    convert = TRUE, exceed.Ttr = FALSE, exceed.rhomin = FALSE,
    logact = NULL, autobalance = TRUE, IS = 0)
```

In [ ]:
def subcrt(species, coeff=None, state=None,
           property=["logK","G","H","S","V","Cp"],
           T=None, P=None, grid=None,
           convert=True, exceed_Ttr=False, exceed_rhomin=False,
           logact=None, autobalance=True, IS=None):
    
    """
    Python wrapper for the subcrt() function in CHNOSZ, an R package created by Dr. Jeffrey Dick.
    """
    
    if not isinstance(species, list):
        args = {'species':species}
        single_species = True
    else:
        args = {'species':ro.StrVector(species)}
        single_species = False
        
    if coeff != None: args["coeff"] = ro.FloatVector(coeff)
        
    if state != None:
        if not isinstance(state, list):
            args["state"] = state
        else:
            args["state"] = ro.StrVector(state)
    
    if not isinstance(property, list): property = [property]
    args["property"] = ro.StrVector(property)
    
    if T != None:
        if not isinstance(T, list):
            args['T'] = T
        else:
            args['T'] = ro.FloatVector(T)

    if P != None:
        if not isinstance(P, list):
            args['P'] = P
        else:
            args['P'] = ro.FloatVector(P)
    
    if grid != None: args['grid'] = grid # grid is either 'T' or 'P'
    
    args['convert'] = convert
    args['exceed.Ttr'] = exceed_Ttr
    args['exceed.rhomin'] = exceed_rhomin
    
    if logact != None: args["logact"] = ro.FloatVector(logact)
    
    args['autobalance'] = autobalance
    
    if IS != None:
        if not isinstance(IS, list):
            args["IS"] = IS
        else:
            args["IS"] = ro.FloatVector(IS)
    
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always")
        a = CHNOSZ.subcrt(**args)
    
    for warning in w:
        print(warning.message)
    
    if len(a) == 3:
        warn = a[2][0] # subcrt's list includes warnings only if they appear
    else:
        warn = None
    
    if single_species:
        out_dict = {"species":pandas2ri.ri2py_dataframe(a[0]),
                    "out":pandas2ri.ri2py_dataframe(a[1][0])} # the extra [0] is important
    else:
        out_dict = {"reaction":pandas2ri.ri2py_dataframe(a[0]),
                    "out":pandas2ri.ri2py_dataframe(a[1])}
        
    if warn != None:
        out_dict["warnings"] = warn
    
    return out_dict

out = subcrt(["H2", "O2", "H2O"], [-1.0, -0.5, 1.0], ["aq", "gas", "liq"],
             "logK", T=[30, 50], P=100, logact=[-4, -3, -5], IS=0.1)

#out = subcrt("CH4")
out

In [ ]:
out["reaction"]
#out["species"]

In [ ]:
out["out"]

In [ ]:
out["warnings"]

In [ ]:
subcrt(species=["H2O"], coeff=[1])["warnings"]

In [ ]:
# TODO: figure out optional arguments.